<a href="https://colab.research.google.com/github/pschorey/Valpo_IT533/blob/main/04_Pedagogy_Web_Scraping_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time #used to slow down scraping so we don't get blocked etc.
import re
%matplotlib inline

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [4]:
from google.colab import drive #access to Google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [41]:
#already saved, no need to run again...
#get url's for the top 1000 songs on letssingit website

baseURL = "https://www.letssingit.com/songs/popular/"
urlList = [] #should be href to top 1000 songs
for i in range(1,21): #loop 1-20
  currentURL = f"{baseURL}{i}" #add number to the end of url string
  html = urlopen(currentURL) #get contents of current url
  soup = BeautifulSoup(html, 'lxml') #parse retrieved html as soup obj
  pageLinks = soup.find_all("a", class_="high_profile") #creates a list of <a class="high_profile"> tags on current page
  for link in pageLinks:
    regex = 'http.*?">' #starting with http and one ore more chars following until first ">
    x = re.findall(regex, str(link)) #create list of all url unformatted strings on current page
    for src in x:
      formSrc = src[:-2] #removes last two chars from link
      urlList.append(formSrc)
  print(len(urlList))
  time.sleep(10) #wait 10 secs before going to next page

for x in urlList:
  print(x)


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
https://www.letssingit.com/miley-cyrus-lyrics-flowers-d7nsn9k
https://www.letssingit.com/ed-sheeran-lyrics-eyes-closed-kd3k3kr
https://www.letssingit.com/harry-styles-lyrics-as-it-was-mxlcs21
https://www.letssingit.com/metro-boomin-the-weeknd-and-21-savage-lyrics-creepin-7bqlg31
https://www.letssingit.com/post-malone-lyrics-chemical-z6wjv61
https://www.letssingit.com/onerepublic-lyrics-i-ain-t-worried-39qfpwl
https://www.letssingit.com/lewis-capaldi-lyrics-wish-you-the-best-vcwq9r3
https://www.letssingit.com/alessandra-lyrics-queen-of-kings-28jtlt8
https://www.letssingit.com/k%C3%A4%C3%A4rij%C3%A4-lyrics-cha-cha-cha-dbv5dss
https://www.letssingit.com/p-nk-lyrics-trustfall-d6m51lq
https://www.letssingit.com/the-weeknd-lyrics-die-for-you-mklp5b9
https://www.letssingit.com/taylor-swift-lyrics-karma-gnjf19q
https://www.letssingit.com/luke-combs-lyrics-fast-car-47vs34w
https://www.letssingit.com/eslabon-armado-f

In [45]:
#already saved, no need to run again
#create df of sources

sourcesDF = pd.DataFrame(urlList)

#Save sourcesDF to google drive, so we don't have to keep scraping the website
path = '/content/drive/My Drive/sourcesDF.csv'#code from: https://cyublog.com/articles/python-en/colab-pandas-three-ways-to-save-dataframe-data/
with open(path, 'w', encoding = 'utf-8-sig') as f:
  sourcesDF.to_csv(f)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#open saved sourcesDF
sourcesDF = pd.read_csv('/content/drive/My Drive/sourcesDF.csv')
print(sourcesDF.head())

   Unnamed: 0                                                  0
0           0  https://www.letssingit.com/miley-cyrus-lyrics-...
1           1  https://www.letssingit.com/ed-sheeran-lyrics-e...
2           2  https://www.letssingit.com/harry-styles-lyrics...
3           3  https://www.letssingit.com/metro-boomin-the-we...
4           4  https://www.letssingit.com/post-malone-lyrics-...


In [ ]:

#loop through every song lyrics page
allSongsData = []
row = 0
while row < 1000: #rows 0-999
  url = sourcesDF.iloc[row,1]
  html = urlopen(url) #get contents of current song url
  soup = BeautifulSoup(html, 'lxml') #parse retrieved html as soup obj

  #get and format artist name
  title = soup.title
  regex = '>.*#? -'
  x = re.findall(regex, str(title))
  artist = x[0][1:-2]
  print(artist)

  #get and format song name
  regex = '- .*? Lyrics'
  x = re.findall(regex, str(title))
  songTitle = x[0][2:-7]
  print(songTitle)

  #get and format lyrics
  lyrics = soup.find("div", id="lyrics")
  song = lyrics.get_text()
  song = song.replace(r'\n',' ')
  print(song)

  allSongsData.append([artist, songTitle, song]) #add details to master array

  row += 1
  time.sleep(5) #wait 5 secs before going to next song page

print('All Done!')



In [ ]:
songDataDF = pd.DataFrame(allSongsData)
songDataDF.columns = ['Artist', 'Song', 'Lyrics']
songDataDF.head()

In [ ]:
#save all song data to drive
path = '/content/drive/My Drive/songDataDF.csv'#code from: https://cyublog.com/articles/python-en/colab-pandas-three-ways-to-save-dataframe-data/
with open(path, 'w', encoding = 'utf-8-sig') as f:
  songDataDF.to_csv(f)

In [ ]:
#open songDataDF from drive
songDataDF = pd.read_csv('/content/drive/My Drive/songDataDF.csv')
print(songDataDF.head())